# Comparing values in single columns between two Pandas dataframes

One of your DataFrames has a column of values. A second DataFrame has a volumn of similar values. How many values do these columns have in common?

Depending on your goal (merging, de-duplicating, identifying index positions of non-similar values), multiple solutions exist. h/t [Chris Albon](https://chrisalbon.com/python/data_wrangling/pandas_join_merge_dataframe/) for data.

## Create two DataFrames

One with null values. Note: a couple employer values overlap (NaN, Facebook).

In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5'],
        'first_name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'], 
        'last_name': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches'],
        'employer': ['Facebook', 'Google', 'Amazon', 'Apple', np.nan]}

df_a = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name', 'employer'])
df_a

,subject_id,first_name,last_name,employer
0,1,Alex,Anderson,Facebook
1,2,Amy,Ackerman,Google
2,3,Allen,Ali,Amazon
3,4,Alice,Aoni,Apple
4,5,Ayoung,Atiches,NaN


In [3]:
raw_data = {
        'subject_id': ['4', '5', '6', '7', '8'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'], 
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan'],
        'employer': [np.nan, 'Kensho', 'Robinhood', 'LendUp', 'Facebook']}
df_b = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name', 'employer'])
df_b

,subject_id,first_name,last_name,employer
0,4,Billy,Bonder,NaN
1,5,Brian,Black,Kensho
2,6,Bran,Balwner,Robinhood
3,7,Bryce,Brice,LendUp
4,8,Betty,Btisan,Facebook


## Identify that duplicates exist
First, we need to show duplicate employers between our dataframes exist.

In [4]:
# Use .all(), a numpy method
df_a.employer.all() == df_b.employer.all()

False

In [5]:
# The indices among these two *ARE* identical 
df_a.index.all() == df_b.index.all()

True

## Combine our dataframes
Now, we seek to join together our two dataframes. Note there are many ways ([join, merge, concatenate, append](https://pandas.pydata.org/pandas-docs/stable/merging.html)) to do this, depending on our desired result.

In our case, we seek a single dataframe where the column IDs are shared among the two initial dataframes. This is what append does.

Once we have a single dataframe with shared column values, we will handle de-duping the 'employer' column.


In [6]:
# append DataFrame B onto DataFram A; store in 'result'
result = df_a.append(df_b)

In [7]:
# show 'result' - note we still have duplicate employer values
result

,subject_id,first_name,last_name,employer
0,1,Alex,Anderson,Facebook
1,2,Amy,Ackerman,Google
2,3,Allen,Ali,Amazon
3,4,Alice,Aoni,Apple
4,5,Ayoung,Atiches,NaN
0,4,Billy,Bonder,NaN
1,5,Brian,Black,Kensho
2,6,Bran,Balwner,Robinhood
3,7,Bryce,Brice,LendUp
4,8,Betty,Btisan,Facebook


In [8]:
# As an aside, this is what merge would have done
df_a.merge(df_b, left_on='employer', right_on='employer', )

,subject_id_x,first_name_x,last_name_x,employer,subject_id_y,first_name_y,last_name_y
0,1,Alex,Anderson,Facebook,8,Betty,Btisan
1,5,Ayoung,Atiches,NaN,4,Billy,Bonder


In [9]:
# As an aside, this is what merge would have done
pd.concat([df_a, df_b], axis=1, join='inner')

,subject_id,first_name,last_name,employer,subject_id,first_name,last_name,employer
0,1,Alex,Anderson,Facebook,4,Billy,Bonder,NaN
1,2,Amy,Ackerman,Google,5,Brian,Black,Kensho
2,3,Allen,Ali,Amazon,6,Bran,Balwner,Robinhood
3,4,Alice,Aoni,Apple,7,Bryce,Brice,LendUp
4,5,Ayoung,Atiches,NaN,8,Betty,Btisan,Facebook


## Identify duplicates, and drop them
We can now identify which employer values are duplicated in our dataframe. We will subsequently drop these values.

In [10]:
# produce a Boolean series revealing which rows are duplicated
result.duplicated(subset='employer')

0    False
1    False
2    False
3    False
4    False
0     True
1    False
2    False
3    False
4     True
dtype: bool

In [11]:
# use that Boolean series to filter our dataframe - these are the duplicate employers
result[result.duplicated(subset='employer')]

,subject_id,first_name,last_name,employer
0,4,Billy,Bonder,NaN
4,8,Betty,Btisan,Facebook


In [12]:
# drop duplicates in the employer column
result.drop_duplicates(subset = 'employer', inplace=True)

## tl;dr - Summary
Assuming you have two dataframes, we can do all the above in a single cell. Append them, and drop duplicates in the employer column.

In [13]:
# append two DataFrames; drop duplicates in the employer
tldr = df_a.append(df_b)
tldr.drop_duplicates(subset = 'employer', inplace=True)
tldr

,subject_id,first_name,last_name,employer
0,1,Alex,Anderson,Facebook
1,2,Amy,Ackerman,Google
2,3,Allen,Ali,Amazon
3,4,Alice,Aoni,Apple
4,5,Ayoung,Atiches,NaN
1,5,Brian,Black,Kensho
2,6,Bran,Balwner,Robinhood
3,7,Bryce,Brice,LendUp


*Ask me questions on Twitter: [@josephofiowa](https://twitter.com/josephofiowa)*